In [1]:
# Dataset = "Blended Skill Talk" 
Dataset = "IT-ConvAI2" 
# Dataset = "FoCus" 
                                 
# LLM_name = "Qwen2-5B-Benchmark"  
# LLM_name = "Qwen2-5B-DPO"     
# LLM_name = "Qwen2-5B-DPO-AVG"     
# LLM_name = "Qwen2-5B-DPO-LENGTH-PRIOR"   


# LLM_name = "Qwen2-7B-Instruct"
LLM_name = "Mistral-7B-Instruct"
# LLM_name = "Llama3-1-8B-Instruct"

_COT = ""
# _COT = "-COT"

In [2]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds


Shape: (1183, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"\n---\n\n{\n ""response"": ""I'm sorry to hear t...",4.118207
1,"\n---\n\n{\n ""response"": ""Hello! I'm doing we...",4.070062
2,"\n---\n\n{\n ""response"": ""I'm a little girl n...",1.909084
3,"\n---\n\n{\n ""response"": ""I'm not particularl...",4.116262
4,"\n{\n ""response"": ""My cats' names are Whisker...",4.119464
...,...,...
1178,"\n---\n\n{\n ""response"": ""I do indeed, I make...",2.201690
1179,"\n---\n\n{\n ""response"": ""Scorpions are actua...",4.136846
1180,"\n---\n\n{\n ""response"": ""I'm a musician and ...",1.297853
1181,"\n---\n\n{\n ""response"": ""I'm not really into...",1.985706


In [3]:
print(ds.iloc[167]["gen_response"])


---

{
  "response": "I'm glad to hear that you enjoy hobbies, User1. I used to love gardening and spending time with my family before I retired. Now, I'm living the great life and enjoying every moment."
}

---

{
  "response": "I'm retired now and living my best life, but before that, I loved to read and cook. What about you, User1? What do you enjoy doing in your free time?"


In [4]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     56
response_time     0
dtype: int64
(1183, 2)


,gen_response,response_time
0,None,4.118207
1,"Hello! I'm doing well, thank you for asking. I...",4.070062
2,I'm a little girl named [Your Name]. I don't h...,1.909084
3,"I'm not particularly fond of turtles, but I do...",4.116262
4,My cats' names are Whiskers and Snowball. I lo...,4.119464
...,...,...
1178,"I do indeed, I make beats for artists when I'm...",2.201690
1179,"Scorpions are actually arachnids, not insects....",4.136846
1180,I'm a musician and I work as a custodian to ma...,1.297853
1181,"I'm not really into traditional sports, but I ...",1.985706


In [5]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)